In [25]:
# Import Libraries 

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep
import re

In [23]:
# URL to be webscraped
url = "https://www.careerjet.com.mt/search/jobs?s=&l=Malta"


In [27]:
# Open the URl by web browser (Used this command to make sure to have the updated chrome and webdriver) 
wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get(url)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kristinaxuereb/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [28]:
## Total jobs available in Malta
no_of_jobs = wd.find_element_by_css_selector("p span").get_attribute("innerText")[:-5]
no_of_jobs = int(no_of_jobs.replace(',', ''))
no_of_jobs

4305

In [29]:
## Getting the first link and clicking on it: 

first_job = wd.find_element_by_class_name('job')
first_job_link = first_job.find_element_by_tag_name('a').click()

## Now the driver is ready to scrape the first job listing

In [30]:
## Now we begin the actual scraping of the data. Extracted what was actually important for analysis. 

title_list = []
company_name_list = []
contract_type_list = []
working_hours_list = []
description_list = []

for i in range(0, no_of_jobs-1):
    try:
        title = wd.find_element_by_xpath("//*[@id=\"job\"]/div/header/h1").get_attribute("innerText")
    except:
        title = "None"
    title_list.append(title)
    try:
        company_name = wd.find_element_by_xpath("//*[@id=\"job\"]/div/header/p").text
    except:
        company_name = "None"
    company_name_list.append(company_name)
    try:
        contract_type = wd.find_element_by_xpath("//*[@id=\"job\"]/div/header/ul[1]/li[2]").text
    except:
        contract_type = "None"
    contract_type_list.append(contract_type)
    try:
        working_hours = wd.find_element_by_xpath("//*[@id=\"job\"]/div/header/ul[1]/li[3]").text
    except:
        working_hours = "None"
    working_hours_list.append(working_hours)
    try:
        description = wd.find_element_by_class_name('content').text
    except:
        description = "None"
    description_list.append(description)
    
    current_url = wd.current_url
    
    next_page = wd.find_element_by_class_name('next').click()
    wait = WebDriverWait(wd,10)
    wait.until(lambda x: x.current_url != current_url)
    

In [31]:
#Finding the length of the list

len(description_list)

4304

 ### Data Description and Data Validation

In [32]:

import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk import tokenize
from nltk import sent_tokenize,word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer


In [33]:
# Transforming the data into a df:

dictionary = {'Job_Title': title_list,
             'Company_Name':company_name_list,
             'Contract_Type': contract_type_list,
             'Working_Hours': working_hours_list,
              'Description': description_list
             }

df = pd.DataFrame(dictionary)
df

,Job_Title,Company_Name,Contract_Type,Working_Hours,Description
0,Dental Assistant,Misco,Permanent,Full-time,Job Description\n\nClosing Date - 30/06/2021\n...
1,Receptionist,Misco,Permanent,Full-time,Job Description\n\nClosing Date - 15/07/2020\n...
2,CLEANER,deVere & Partners Holding Ltd,Permanent,Full-time,Responsibilities & Requisites: \nMust be comfo...
3,chef de partie,NAAR Restobar,"€30,000 per year",Permanent,Work in a dynamic and cosmopolitan restaurant ...
4,Twitch Streamer (French Language),Blue Window Ltd,Contract,Part-time,Do you want to join a dynamic team and become ...
...,...,...,...,...,...
4299,Process Operator,None,Permanent,Full-time,Process Operator\n\nPlaymobil Malta Ltd\n\nVac...
4300,Mobile Crane Driver,None,Permanent,Full-time,Mobile Crane Driver\n\nB&B Construction Ltd.\n...
4301,REAL ESTATE AGENT,None,Permanent,Full-time,REAL ESTATE AGENT\n\nQuicklets Gzira\n\nReal E...
4302,Delivery Person/ Helper,None,Permanent,Part-time,Delivery Person/ Helper\n\nCJ Transport\n\nFur...


In [34]:
# Data types
df.dtypes

Job_Title        object
Company_Name     object
Contract_Type    object
Working_Hours    object
Description      object
dtype: object

In [35]:
# Checking if I have any missing values
df.isna().sum()

Job_Title        0
Company_Name     0
Contract_Type    0
Working_Hours    0
Description      0
dtype: int64

## Cleaning the data

In [36]:
# Text is converted to lower_case and all special characters are removed

def list_tokenizer(list):
    clean = [re.sub('[^A-Za-z0-9]+', ' ',i).lower() for i in list]
    sentences_tokens = [sent_tokenize(i) for i in clean]
    word_tokens = [word_tokenize(i) for i in clean]
    return word_tokens


In [37]:
clean_description = list_tokenizer(description_list)
clean_title = list_tokenizer(title_list)

In [ ]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [38]:
tagged_tuples = [pos_tag(element, tagset='universal') for element in clean_description]